In [22]:
import os
import re
import time
import json
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

load_dotenv()
# WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

kakao_id = os.getenv("KAKAO_ID")
kakao_pw = os.getenv("KAKAO_PWD")
driver.get("https://webtoon.kakao.com/more")

# 로그인처리
login_icon = r"#root > main > div > div.bg-background-02 > div > div > a:nth-child(1) > div > img.w-113.h-38"
driver.find_element(By.CSS_SELECTOR, login_icon).click()
time.sleep(2)
login_button = r"body > div:nth-child(5) > div > div > div > div.overflow-x-hidden.overflow-y-auto.\!overflow-hidden.flex.flex-col > div.text-center.pb-\[112px\].overflow-y-auto > div > div > button"
driver.find_element(By.CSS_SELECTOR, login_button).click()
time.sleep(2)

ID_selector = r"#loginId--1"
PWD_selector = r"#password--2"
ID_input = driver.find_element(By.CSS_SELECTOR, ID_selector)
ActionChains(driver).send_keys_to_element(ID_input, kakao_id).perform()
PWD_input = driver.find_element(By.CSS_SELECTOR, PWD_selector)
ActionChains(driver).send_keys_to_element(PWD_input, kakao_pw).perform()
login_selector = (
    r"#mainContent > div > div > form > div.confirm_btn > button.btn_g.highlight.submit"
)
driver.find_element(By.CSS_SELECTOR, login_selector).click()
time.sleep(2)
# 동시접속 기기 횟수 초과 시

alert = r"body > div:nth-child(8) > div > div > div > div.absolute.w-full.px-18.bottom-0.pb-30.flex.left-0.z-1.bg-grey-01.light\:bg-white.Alert_buttonsWrap__2ln9d.pt-10 > button.relative.px-10.py-0.btn-white.light\:btn-black.button-active"
driver.find_element(By.CSS_SELECTOR, alert).click()


In [ ]:
import json
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ✅ 기존 JSON 데이터 불러오기
json_file = "data/webtoon_kko/webtoon_kko_crawling.json"
with open(json_file, "r", encoding="utf-8") as f:
    webtoon_data = json.load(f)

# ✅ 카카오 웹툰 페이지 열기
url = "https://webtoon.kakao.com"
driver.get(url)
time.sleep(1)  # 페이지 로딩 대기

# ✅ 요일 버튼 가져오기(월 ~ 완결, 신작 제외)
day_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.relative.z-navigationBarMinus1.mx-auto.my-0.w-full > div.w-full.h-full.relative > ul > li:nth-child({}) > p"
total_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.w-full.h-40.sticky.flex.px-11.border-t-1.pb-10.justify-center.overflow-hidden > div > button:nth-child(1)"

# def scroll_to_bottom():
#     """페이지 끝까지 스크롤하여 모든 웹툰을 로드"""
#     last_height = driver.execute_script("return document.body.scrollHeight")
#     while True:
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(5)  # 스크롤 후 데이터 로딩 대기
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:  # 스크롤이 더 이상 내려가지 않으면 종료
#             break
#         last_height = new_height

# def scroll_to_element(driver, element):
#     """요소가 화면에 보이도록 스크롤"""
#     WebDriverWait(driver, 2).until(EC.visibility_of(element))
#     driver.execute_script("arguments[0].scrollIntoView(true);", element)
#     time.sleep(1)  # 스크롤 후 페이지 로딩 대기

def slow_scroll(driver):
    """천천히 스크롤 다운 (400px씩)"""
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollBy(0, 400);")  # 400px씩 스크롤
        time.sleep(1)  # 1초 대기

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 더 이상 스크롤이 안되면 종료
        last_height = new_height

# ✅ 썸네일 정보 저장할 딕셔너리
webtoon_thumbnails  = {}

# ✅ 카테고리별 웹툰 크롤링
for id in range(2, 10):  # 월~완결 (신작 제외)
    category_selector = day_sel.format(id)
    category = driver.find_element(By.CSS_SELECTOR, category_selector)
    total = driver.find_element(By.CSS_SELECTOR, total_sel)
    category.click()
    time.sleep(1)
    total.click()
    time.sleep(1)
    
    # ✅ 웹툰 리스트 가져오기 (점진적으로 스크롤)
    last_count = 0
    while True:
        webtoons = driver.find_elements(By.CSS_SELECTOR, "a[href*='/content/']")
        
        if len(webtoons) == last_count:
            break  # 더 이상 로딩되지 않으면 종료

        last_count = len(webtoons)

        # # 마지막 웹툰 요소로 스크롤하여 추가 데이터 로드 유도
        # scroll_to_element(driver, webtoons[-1])
        slow_scroll(driver)
    # ✅ 각 웹툰의 ID 및 썸네일 크롤링
    for webtoon in webtoons:
        webtoon_url = webtoon.get_attribute("href")

        # ✅ 웹툰 ID 추출
        match = re.search(r'/content/.+/(\d+)', webtoon_url)
        if match:
            webtoon_id = match.group(1)  # 숫자 ID 추출
        else:
            continue

        # ✅ 썸네일 이미지 URL 가져오기
        try:
            thumbnail_element = webtoon.find_element(By.CSS_SELECTOR, "img.w-full.h-full.object-cover.object-top")
            thumbnail = thumbnail_element.get_attribute("src")
        except:
            thumbnail = ""  # 썸네일이 없는 경우 빈 문자열

        # ✅ ID 기준으로 썸네일 정보 저장
        webtoon_thumbnails[webtoon_id] = thumbnail

# ✅ 기존 데이터와 썸네일 정보 병합
for webtoon in webtoon_data:
    webtoon_id = str(webtoon["id"])  # ID를 문자열로 변환
    if webtoon_id in webtoon_thumbnails:
        webtoon["thumbnail"] = webtoon_thumbnails[webtoon_id]  # ✅ 기존 데이터에 썸네일 추가

# ✅ JSON 파일 저장
output_file = "webtoon_kko_crawling_with_thumbnails.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(webtoon_data, f, ensure_ascii=False, indent=4)

print(f"✅ 썸네일 추가 완료! {output_file} 파일로 저장됨.")

<>:22: SyntaxWarning: invalid escape sequence '\:'
<>:23: SyntaxWarning: invalid escape sequence '\:'
<>:22: SyntaxWarning: invalid escape sequence '\:'
<>:23: SyntaxWarning: invalid escape sequence '\:'
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16352\1842128082.py:22: SyntaxWarning: invalid escape sequence '\:'
  day_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\s\].bg-background > div.relative.z-navigationBarMinus1.mx-auto.my-0.w-full > div.w-full.h-full.relative > ul > li:nth-child({}) > p"
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16352\1842128082.py:23: SyntaxWarning: invalid escape sequence '\:'
  total_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\s\].bg-background > div.w-full.h-40.sticky.flex.px-11.border-t-1.pb-10.justify-ce

In [6]:
import json
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ✅ 기존 JSON 파일 불러오기
with open("data/webtoon_kko/webtoon_kko_crawling.json", "r", encoding="utf-8") as f:
    webtoon_data = json.load(f)  # 기존 크롤링 데이터

# ✅ Selenium WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://webtoon.kakao.com")
time.sleep(2)  # 페이지 로딩 대기

# ✅ 요일별 카테고리 선택 (월 ~ 완결)
day_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.relative.z-navigationBarMinus1.mx-auto.my-0.w-full > div.w-full.h-full.relative > ul > li:nth-child({}) > p"
total_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.w-full.h-40.sticky.flex.px-11.border-t-1.pb-10.justify-center.overflow-hidden > div > button:nth-child(1)"

def scroll_to_element(driver, element):
    """요소가 화면에 보이도록 스크롤"""
    WebDriverWait(driver, 5).until(EC.visibility_of(element))
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(2)  # 스크롤 후 페이지 로딩 대기

# ✅ 전체 웹툰 ID-썸네일 정보 저장할 딕셔너리
webtoon_thumbnails = {}

# ✅ 카테고리별 웹툰 크롤링 (월~완결)
for id in range(2, 10):  # 월~완결 (신작 제외)
    category_selector = day_sel.format(id)
    category = driver.find_element(By.CSS_SELECTOR, category_selector)
    total = driver.find_element(By.CSS_SELECTOR, total_sel)
    category.click()
    time.sleep(1)
    total.click()
    time.sleep(1)

    # ✅ 웹툰 리스트 가져오기
    webtoons = driver.find_elements(By.CSS_SELECTOR, "a[href*='/content/']")

    # ✅ 각 웹툰의 ID 및 썸네일 크롤링
    for webtoon in webtoons:
        scroll_to_element(driver, webtoon)  # 🔹 웹툰 요소로 스크롤
        time.sleep(1)  # 데이터 로딩 대기

        webtoon_url = webtoon.get_attribute("href")

        # ✅ 웹툰 ID 추출
        match = re.search(r'/content/.+/(\d+)', webtoon_url)
        if match:
            webtoon_id = match.group(1)  # 숫자 ID 추출
        else:
            continue

        # ✅ 썸네일 이미지 URL 가져오기
        try:
            thumbnail_element = webtoon.find_element(By.CSS_SELECTOR, "img.w-full.h-full.object-cover.object-top")
            thumbnail = thumbnail_element.get_attribute("src")
        except:
            thumbnail = ""  # 썸네일이 없는 경우 빈 문자열

        # ✅ ID 기준으로 썸네일 정보 저장
        webtoon_thumbnails[webtoon_id] = thumbnail

# ✅ 기존 데이터와 썸네일 정보 병합
for webtoon in webtoon_data:
    webtoon_id = str(webtoon["id"])  # ID를 문자열로 변환
    if webtoon_id in webtoon_thumbnails:
        webtoon["thumbnail"] = webtoon_thumbnails[webtoon_id]  # ✅ 기존 데이터에 썸네일 추가

# ✅ JSON 파일 저장
output_file = "webtoon_kko_crawling_with_thumbnails.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(webtoon_data, f, ensure_ascii=False, indent=4)

print(f"✅ 썸네일 추가 완료! {output_file} 파일로 저장됨.")

# ✅ 드라이버 종료
driver.quit()


<>:21: SyntaxWarning: invalid escape sequence '\:'
<>:22: SyntaxWarning: invalid escape sequence '\:'
<>:21: SyntaxWarning: invalid escape sequence '\:'
<>:22: SyntaxWarning: invalid escape sequence '\:'
C:\Users\Playdata\AppData\Local\Temp\ipykernel_23556\617811312.py:21: SyntaxWarning: invalid escape sequence '\:'
  day_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\s\].bg-background > div.relative.z-navigationBarMinus1.mx-auto.my-0.w-full > div.w-full.h-full.relative > ul > li:nth-child({}) > p"
C:\Users\Playdata\AppData\Local\Temp\ipykernel_23556\617811312.py:22: SyntaxWarning: invalid escape sequence '\:'
  total_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\s\].bg-background > div.w-full.h-40.sticky.flex.px-11.border-t-1.pb-10.justify-cent

TimeoutException: Message: 


In [ ]:
import json
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ✅ 기존 JSON 데이터 불러오기
json_file = "data/webtoon_kko/webtoon_kko_crawling.json"
with open(json_file, "r", encoding="utf-8") as f:
    webtoon_data = json.load(f)

# ✅ 카카오 웹툰 페이지 열기
url = "https://webtoon.kakao.com"
driver.get(url)
time.sleep(1)  # 페이지 로딩 대기

# ✅ 요일 버튼 가져오기(월 ~ 완결, 신작 제외)
day_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.relative.z-navigationBarMinus1.mx-auto.my-0.w-full > div.w-full.h-full.relative > ul > li:nth-child({}) > p"
total_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.w-full.h-40.sticky.flex.px-11.border-t-1.pb-10.justify-center.overflow-hidden > div > button:nth-child(1)"

def smart_scroll(driver, scroll_step=300, pause_time=1):
    """스크롤을 여러 번 나누어서 내려가면서, 데이터가 로딩될 시간을 주는 함수"""
    last_height = driver.execute_script("return document.body.scrollHeight")  # 페이지 전체 높이
    new_height = 0

    while new_height < last_height:
        driver.execute_script(f"window.scrollBy(0, {scroll_step});")  # 지정한 만큼 스크롤
        time.sleep(pause_time)  # 대기 (데이터 로딩을 기다림)

        new_height = driver.execute_script("return window.scrollY")  # 현재 스크롤 위치
        if new_height + scroll_step >= last_height:  # 끝까지 스크롤했을 경우 종료
            break


# ✅ 썸네일 정보 저장할 딕셔너리
webtoon_thumbnails  = {}

# ✅ 카테고리별 웹툰 크롤링
for id in range(2, 10):  # 월~완결 (신작 제외)
    category_selector = day_sel.format(id)
    category = driver.find_element(By.CSS_SELECTOR, category_selector)
    total = driver.find_element(By.CSS_SELECTOR, total_sel)
    category.click()
    time.sleep(1)
    total.click()
    time.sleep(1)
    
    
    # ✅ 웹툰 리스트 가져오기 (점진적으로 스크롤)
    last_count = 0
    while True:
        webtoons = driver.find_elements(By.CSS_SELECTOR, "a[href*='/content/']")
        
        if len(webtoons) == last_count:
            break  # 더 이상 로딩되지 않으면 종료

        last_count = len(webtoons)

        # # 마지막 웹툰 요소로 스크롤하여 추가 데이터 로드 유도
        # scroll_to_element(driver, webtoons[-1])

        slow_scroll(driver)

    # ✅ 각 웹툰의 ID 및 썸네일 크롤링
    for webtoon in webtoons:
        webtoon_url = webtoon.get_attribute("href")

        # ✅ 웹툰 ID 추출
        match = re.search(r'/content/.+/(\d+)', webtoon_url)
        if match:
            webtoon_id = match.group(1)  # 숫자 ID 추출
        else:
            continue

        # ✅ 썸네일 이미지 URL 가져오기
        try:
            thumbnail_element = webtoon.find_element(By.CSS_SELECTOR, "img.w-full.h-full.object-cover.object-top")
            thumbnail = thumbnail_element.get_attribute("src")
        except:
            thumbnail = ""  # 썸네일이 없는 경우 빈 문자열

        # ✅ ID 기준으로 썸네일 정보 저장
        webtoon_thumbnails[webtoon_id] = thumbnail

# ✅ 기존 데이터와 썸네일 정보 병합
for webtoon in webtoon_data:
    webtoon_id = str(webtoon["id"])  # ID를 문자열로 변환
    if webtoon_id in webtoon_thumbnails:
        webtoon["thumbnail"] = webtoon_thumbnails[webtoon_id]  # ✅ 기존 데이터에 썸네일 추가

# ✅ JSON 파일 저장
output_file = "webtoon_kko_crawling_with_thumbnails.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(webtoon_data, f, ensure_ascii=False, indent=4)

print(f"✅ 썸네일 추가 완료! {output_file} 파일로 저장됨.")

In [17]:
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs

# ✅ Selenium WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# ✅ 카카오 웹툰 페이지 열기
url = "https://webtoon.kakao.com"
driver.get(url)
time.sleep(2)  # 페이지 로딩 대기

# ✅ 기존 JSON 파일 로드
with open("data/webtoon_kko/webtoon_kko_crawling.json", "r", encoding="utf-8") as f:
    webtoon_data = json.load(f)

# ✅ 요일별 웹툰 크롤링 설정
day_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.relative.z-navigationBarMinus1.mx-auto.my-0.w-full > div.w-full.h-full.relative > ul > li:nth-child({}) > p"
total_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.w-full.h-40.sticky.flex.px-11.border-t-1.pb-10.justify-center.overflow-hidden > div > button:nth-child(1)"

# ✅ # ✅ 스크롤을 내려 페이지 로딩을 유도하는 함수
def scroll_to_bottom(driver):
    scroll_pause_time = 1.5  # 스크롤 후 대기 시간
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 바닥까지 스크롤
        time.sleep(scroll_pause_time)  # 페이지 로딩 대기

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # 새로 로드된 데이터가 없으면 종료
            break
        last_height = new_height

# ✅ 마지막 웹툰의 인덱스를 찾는 함수
def get_last_webtoon_index(driver):
    webtoons = driver.find_elements(By.CSS_SELECTOR, 
        "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\\:w-default-max-width.md\\:w-\\[490px\\] > div.flex.flex-col.gap-4 > div.flex.flex-wrap.gap-4.content-start > div")
    
    last_index = len(webtoons)  # 마지막 웹툰 요소 개수 = nth-child(X) 값
    print(f"✅ 마지막 웹툰의 인덱스: {last_index}")
    return last_index

# ✅ 모든 웹툰 썸네일을 가져오는 함수
def get_all_webtoon_thumbnails(driver, last_index):
    thumbnails = []
    
    for i in range(1, last_index + 1):  # 1번째부터 마지막 nth-child까지 반복
        selector = f"#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\\:w-default-max-width.md\\:w-\\[490px\\] > div.flex.flex-col.gap-4 > div.flex.flex-wrap.gap-4.content-start > div:nth-child({i}) > div > div > a > picture:nth-child(2) > img"
        
        try:
            img_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, selector))
            )
            img_url = img_element.get_attribute("src")
            thumbnails.append({"index": i, "image_url": img_url})
            print(f"✅ {i}번째 썸네일 가져옴: {img_url}")
        
        except Exception as e:
            print(f"❌ {i}번째 썸네일 로딩 실패: {e}")
    
    return thumbnails

# ✅ 실행 순서
scroll_to_bottom(driver)  # 바닥까지 스크롤
last_index = get_last_webtoon_index(driver)  # 마지막 웹툰 인덱스 찾기
thumbnails = get_all_webtoon_thumbnails(driver, last_index)  # 모든 썸네일 수집

# ✅ 웹툰 ID 추출 함수
def extract_webtoon_id(url):
    """웹툰 URL에서 ID 추출"""
    match = re.search(r'/content/.+/(\d+)', url)
    return int(match.group(1)) if match else None

# ✅ 카테고리별 웹툰 크롤링
thumbnail_dict = {}

for id in range(2, 3):  # 월~완결 (신작 제외)
    category_selector = day_sel.format(id)
    total_selector = total_sel

    category = driver.find_element(By.CSS_SELECTOR, category_selector)
    category.click()
    time.sleep(1)  # ✅ 데이터 로딩 대기

    total = driver.find_element(By.CSS_SELECTOR, total_selector)
    total.click()
    time.sleep(1)  # ✅ 데이터 로딩 대기

    # ✅ 스크롤 다운하여 모든 웹툰 로딩
    time.sleep(1)  # ✅ 데이터가 로딩될 시간을 줌

    # ✅ 웹툰 리스트 가져오기
    soup = bs(driver.page_source, "html.parser")
    webtoons = soup.select("a[href*='/content/']")

    for webtoon in webtoons:
        webtoon_url = webtoon.get("href")
        webtoon_id = extract_webtoon_id(webtoon_url)

        if webtoon_id:
            # ✅ 썸네일 이미지 URL 가져오기
            thumbnail_element = webtoon.select_one("img.w-full.h-full.object-cover.object-top")
            thumbnail = thumbnail_element["src"] if thumbnail_element else ""
            thumbnail_dict[webtoon_id] = thumbnail

# ✅ 기존 JSON 파일과 매칭하여 썸네일 추가
for webtoon in webtoon_data:
    webtoon_id = webtoon["id"]
    if webtoon_id in thumbnail_dict:
        webtoon["thumbnail"] = thumbnail_dict[webtoon_id]

# ✅ JSON 파일 저장
with open("webtoon_kko_crawling_with_thumbnails.json", "w", encoding="utf-8") as f:
    json.dump(webtoon_data, f, ensure_ascii=False, indent=4)

print("✅ 웹툰 썸네일 추가 완료!")

# ✅ WebDriver 종료
driver.quit()


<>:25: SyntaxWarning: invalid escape sequence '\:'
<>:26: SyntaxWarning: invalid escape sequence '\:'
<>:25: SyntaxWarning: invalid escape sequence '\:'
<>:26: SyntaxWarning: invalid escape sequence '\:'
C:\Users\Playdata\AppData\Local\Temp\ipykernel_23556\1270223523.py:25: SyntaxWarning: invalid escape sequence '\:'
  day_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\s\].bg-background > div.relative.z-navigationBarMinus1.mx-auto.my-0.w-full > div.w-full.h-full.relative > ul > li:nth-child({}) > p"
C:\Users\Playdata\AppData\Local\Temp\ipykernel_23556\1270223523.py:26: SyntaxWarning: invalid escape sequence '\:'
  total_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\s\].bg-background > div.w-full.h-40.sticky.flex.px-11.border-t-1.pb-10.justify-ce

📌 더 이상 스크롤할 데이터가 없습니다. 종료합니다.
✅ 웹툰 썸네일 추가 완료!


In [19]:
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs

# ✅ Selenium WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
action = ActionChains(driver)  # ✅ ActionChains 생성

# ✅ 카카오 웹툰 페이지 열기
url = "https://webtoon.kakao.com"
driver.get(url)
time.sleep(2)  # 페이지 로딩 대기

# ✅ 기존 JSON 파일 로드
with open("data/webtoon_kko/webtoon_kko_crawling.json", "r", encoding="utf-8") as f:
    webtoon_data = json.load(f)

# ✅ 요일 버튼 가져오기(월 ~ 완결, 신작 제외)
day_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.relative.z-navigationBarMinus1.mx-auto.my-0.w-full > div.w-full.h-full.relative > ul > li:nth-child({}) > p"
total_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.w-full.h-40.sticky.flex.px-11.border-t-1.pb-10.justify-center.overflow-hidden > div > button:nth-child(1)"

# ✅ 웹툰 ID 추출 함수
def extract_webtoon_id(url):
    """웹툰 URL에서 ID 추출"""
    match = re.search(r'/content/.+/(\d+)', url)
    return int(match.group(1)) if match else None

# ✅ 웹툰 스크롤 및 로딩 함수 (ActionChains 활용)
def scroll_webtoon_page(driver):
    """웹툰 목록을 부드럽게 스크롤하며 모든 데이터를 로딩하는 함수"""
    webtoon_list = driver.find_elements(By.CSS_SELECTOR, 
        "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\\:w-default-max-width.md\\:w-\\[490px\\] > div.flex.flex-col.gap-4 > div.flex.flex-wrap.gap-4.content-start > div")
    
    last_index = len(webtoon_list)
    print(f"✅ 웹툰 목록 개수: {last_index}")

    for i in range(0, last_index, 5):  # ✅ 5개씩 이동하면서 스크롤
        action.move_to_element(webtoon_list[i]).perform()
        time.sleep(1)  # ✅ 스크롤 후 로딩 대기

    return last_index  # ✅ 최종 웹툰 개수 반환

# ✅ 모든 웹툰 썸네일을 가져오는 함수
def get_all_webtoon_thumbnails(driver, last_index):
    """웹툰 ID별 썸네일을 가져와 매칭"""
    thumbnails = {}
    
    for i in range(1, last_index + 1):  # 1번째부터 마지막 nth-child까지 반복
        selector = f"#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\\:w-default-max-width.md\\:w-\\[490px\\] > div.flex.flex-col.gap-4 > div.flex.flex-wrap.gap-4.content-start > div:nth-child({i}) > div > div > a"
        
        try:
            # ✅ 웹툰 링크 및 썸네일 가져오기
            webtoon_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, selector))
            )
            webtoon_url = webtoon_element.get_attribute("href")
            webtoon_id = extract_webtoon_id(webtoon_url)
            
            # ✅ 썸네일 가져오기
            img_element = webtoon_element.find_element(By.CSS_SELECTOR, "picture:nth-child(2) img")
            img_url = img_element.get_attribute("src")

            if webtoon_id:
                thumbnails[webtoon_id] = img_url
                print(f"✅ {webtoon_id}번 썸네일 가져옴: {img_url}")

        except Exception as e:
            print(f"❌ {i}번째 썸네일 로딩 실패: {e}")
    
    return thumbnails

# ✅ 실행 순서
last_index = scroll_webtoon_page(driver)  # ✅ ActionChains로 부드럽게 스크롤
thumbnails = get_all_webtoon_thumbnails(driver, last_index)  # ✅ 썸네일 수집

# ✅ 기존 JSON 파일과 매칭하여 썸네일 추가
for webtoon in webtoon_data:
    webtoon_id = webtoon["id"]
    if webtoon_id in thumbnails:
        webtoon["thumbnail"] = thumbnails[webtoon_id]

# ✅ JSON 파일 저장
with open("webtoon_kko_crawling_with_thumbnails.json", "w", encoding="utf-8") as f:
    json.dump(webtoon_data, f, ensure_ascii=False, indent=4)

print("✅ 웹툰 썸네일 추가 완료!")

# ✅ WebDriver 종료
driver.quit()


✅ 웹툰 목록 개수: 20
✅ 3799번 썸네일 가져옴: https://kr-a.kakaopagecdn.com/P/C/3799/c2/2x/dd2f33ec-6523-4caa-9041-e47a63766d0d.png
✅ 4384번 썸네일 가져옴: https://kr-a.kakaopagecdn.com/P/C/4384/c2/2x/4938b84b-2a05-4be9-b772-2dfea4d4b8aa.png
✅ 1295번 썸네일 가져옴: https://kr-a.kakaopagecdn.com/P/C/1295/c2/2x/b2e2d7a6-abc1-4091-9521-2de46af6c0a6.png
✅ 3014번 썸네일 가져옴: https://kr-a.kakaopagecdn.com/P/C/3014/c2/2x/918fcfdc-d30a-4ebc-ae02-e5e859549b50.png
✅ 4030번 썸네일 가져옴: https://kr-a.kakaopagecdn.com/P/C/4030/c2/2x/288cd440-e75c-4b9d-a4f1-0d06448571f2.png
✅ 3315번 썸네일 가져옴: https://kr-a.kakaopagecdn.com/P/C/3315/c2/2x/4e16189d-2f45-4288-88f8-0a7cdb62c582.png
✅ 4051번 썸네일 가져옴: https://kr-a.kakaopagecdn.com/P/C/4051/c2/2x/01e1a4d1-983c-4294-aab7-b410c74feb20.png
✅ 3929번 썸네일 가져옴: https://kr-a.kakaopagecdn.com/P/C/3929/c2/2x/35edc467-ab8a-4a26-b46f-33dc59a1f0d6.png
✅ 4344번 썸네일 가져옴: https://kr-a.kakaopagecdn.com/P/C/4344/c2/2x/6493ff08-ab9d-43bb-8193-5c2b8763ad19.png
✅ 2970번 썸네일 가져옴: https://kr-a.kakaopagecdn.com/P/C/2970/c2

In [7]:
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs

# ✅ Selenium WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
action = ActionChains(driver)  # ActionChains 생성

# 카카오 웹툰 페이지 열기
url = "https://webtoon.kakao.com"
driver.get(url)
time.sleep(1)  # 페이지 로딩 대기

# 기존 JSON 파일 로드
with open("webtoon_kko_crawling_with_thumbnails.json", "r", encoding="utf-8") as f:
    webtoon_data = json.load(f)

# 웹툰 ID 추출 함수
def extract_webtoon_id(url):
    """웹툰 URL에서 ID 추출"""
    match = re.search(r'/content/.+/(\d+)', url)
    return int(match.group(1)) if match else None

# 점진적 스크롤 함수
def slow_scroll(driver):
    """페이지가 로딩될 수 있도록 천천히 스크롤을 내리는 함수"""
    scroll_pause_time = 0.5
    webtoons = driver.find_elements(By.CSS_SELECTOR, "a[href*='/content/']")

    for webtoon in webtoons:
        try:
            action.move_to_element(webtoon).perform()  # 해당 요소까지 이동
            time.sleep(scroll_pause_time)
        except:
            pass

# 카테고리 & "전체보기" 버튼 선택자
day_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.relative.z-navigationBarMinus1.mx-auto.my-0.w-full > div.w-full.h-full.relative > ul > li:nth-child({}) > p"
total_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\[250ms\].bg-background > div.w-full.h-40.sticky.flex.px-11.border-t-1.pb-10.justify-center.overflow-hidden > div > button:nth-child(1)"

# 웹툰 ID-썸네일 저장 딕셔너리
webtoon_thumbnails = {}

# 카테고리별 웹툰 크롤링
for id in range(9, 10):  # 월~완결 (신작 제외)
    try:
        category_selector = day_sel.format(id)
        category = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, category_selector))
        )
        category.click()
        time.sleep(0.3)

        total = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, total_sel))
        )
        total.click()
        time.sleep(0.3)

        slow_scroll(driver)

        # 웹툰 리스트 가져오기
        webtoons = driver.find_elements(By.CSS_SELECTOR, "a[href*='/content/']")

        # 웹툰 ID & 썸네일 크롤링
        for webtoon in webtoons:
            webtoon_url = webtoon.get_attribute("href")

            # 웹툰 ID 추출
            webtoon_id = extract_webtoon_id(webtoon_url)
            if not webtoon_id:
                continue

            # 썸네일 이미지 URL 가져오기
            try:
                thumbnail_element = webtoon.find_element(By.CSS_SELECTOR, "img.w-full.h-full.object-cover.object-top")
                thumbnail = thumbnail_element.get_attribute("src")
            except:
                thumbnail = ""  # 썸네일이 없는 경우 빈 문자열

            # ID 기준으로 썸네일 정보 저장
            webtoon_thumbnails[webtoon_id] = thumbnail
            print(f"웹툰 {webtoon_id}번 썸네일 저장 완료")

    except Exception as e:
        print(f"❌ 카테고리 {id} 크롤링 중 오류 발생: {e}")

# 기존 JSON 파일과 매칭하여 썸네일 추가
for webtoon in webtoon_data:
    webtoon_id = webtoon["id"]
    if webtoon_id in webtoon_thumbnails:
        webtoon["thumbnail"] = webtoon_thumbnails[webtoon_id]

# JSON 파일 저장
with open("1webtoon_kko_crawling_with_thumbnails.json", "w", encoding="utf-8") as f:
    json.dump(webtoon_data, f, ensure_ascii=False, indent=4)

print("웹툰 썸네일 추가 완료!")

# WebDriver 종료
driver.quit()


<>:46: SyntaxWarning: invalid escape sequence '\:'
<>:47: SyntaxWarning: invalid escape sequence '\:'
<>:46: SyntaxWarning: invalid escape sequence '\:'
<>:47: SyntaxWarning: invalid escape sequence '\:'
C:\Users\Playdata\AppData\Local\Temp\ipykernel_21768\2661598212.py:46: SyntaxWarning: invalid escape sequence '\:'
  day_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\s\].bg-background > div.relative.z-navigationBarMinus1.mx-auto.my-0.w-full > div.w-full.h-full.relative > ul > li:nth-child({}) > p"
C:\Users\Playdata\AppData\Local\Temp\ipykernel_21768\2661598212.py:47: SyntaxWarning: invalid escape sequence '\:'
  total_sel = "#root > main > div > div.px-11.mx-auto.my-0.w-full.lg\:w-default-max-width.md\:w-\[490px\] > div.sticky.w-full.z-navigationBarPlus1.mt-6.transition-all.duration-\s\].bg-background > div.w-full.h-40.sticky.flex.px-11.border-t-1.pb-10.justify-ce

웹툰 4384번 썸네일 저장 완료
웹툰 3155번 썸네일 저장 완료
웹툰 2858번 썸네일 저장 완료
웹툰 2526번 썸네일 저장 완료
웹툰 4173번 썸네일 저장 완료
웹툰 3185번 썸네일 저장 완료
웹툰 3627번 썸네일 저장 완료
웹툰 3845번 썸네일 저장 완료
웹툰 3305번 썸네일 저장 완료
웹툰 4150번 썸네일 저장 완료
웹툰 3973번 썸네일 저장 완료
웹툰 3617번 썸네일 저장 완료
웹툰 3683번 썸네일 저장 완료
웹툰 4058번 썸네일 저장 완료
웹툰 4151번 썸네일 저장 완료
웹툰 3443번 썸네일 저장 완료
웹툰 3916번 썸네일 저장 완료
웹툰 3964번 썸네일 저장 완료
웹툰 3868번 썸네일 저장 완료
웹툰 4212번 썸네일 저장 완료
웹툰 4078번 썸네일 저장 완료
웹툰 3782번 썸네일 저장 완료
웹툰 2760번 썸네일 저장 완료
웹툰 3626번 썸네일 저장 완료
웹툰 3248번 썸네일 저장 완료
웹툰 3246번 썸네일 저장 완료
웹툰 3851번 썸네일 저장 완료
웹툰 4065번 썸네일 저장 완료
웹툰 3469번 썸네일 저장 완료
웹툰 3857번 썸네일 저장 완료
웹툰 3815번 썸네일 저장 완료
웹툰 3539번 썸네일 저장 완료
웹툰 3473번 썸네일 저장 완료
웹툰 3594번 썸네일 저장 완료
웹툰 3976번 썸네일 저장 완료
웹툰 3059번 썸네일 저장 완료
웹툰 2952번 썸네일 저장 완료
웹툰 3089번 썸네일 저장 완료
웹툰 2315번 썸네일 저장 완료
웹툰 3602번 썸네일 저장 완료
웹툰 2804번 썸네일 저장 완료
웹툰 3314번 썸네일 저장 완료
웹툰 4123번 썸네일 저장 완료
웹툰 1816번 썸네일 저장 완료
웹툰 2327번 썸네일 저장 완료
웹툰 3221번 썸네일 저장 완료
웹툰 3006번 썸네일 저장 완료
웹툰 2394번 썸네일 저장 완료
웹툰 3805번 썸네일 저장 완료
웹툰 2900번 썸네일 저장 완료
웹툰 3781번 썸네일 저장 완료
웹툰 3808번 썸네일 저장 완료
웹툰 2888번 썸네일